In [1]:
# https://www.kaggle.com/netssfy/learning-curve
# https://medium.com/i-like-big-data-and-i-cannot-lie/how-i-scored-in-the-top-9-of-kaggles-titanic-machine-learning-challenge-243b5f45c8e9
import re
import pandas as pd
import numpy as np

pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)

In [2]:
dat = pd.read_csv("../local-data/input/train.csv")
dat.shape

(891, 12)

In [3]:
df = dat.copy()

In [4]:
missing = df.copy().isna().sum()
(missing.reset_index(name='cnt')
 .sort_values(by='cnt', ascending=True))

,index,cnt
0,PassengerId,0
1,Survived,0
2,Pclass,0
3,Name,0
4,Sex,0
6,SibSp,0
7,Parch,0
8,Ticket,0
9,Fare,0
11,Embarked,2


In [5]:
df['Cabin'].value_counts(normalize=True).head()

C23 C25 C27    0.019608
G6             0.019608
B96 B98        0.019608
F2             0.014706
C22 C26        0.014706
Name: Cabin, dtype: float64

In [6]:
# modify Cabin
df['Cabin'] = df['Cabin'].fillna('U')
df['Cabin'] = df.Cabin.str.extract('([A-Za-z])', expand=False)

# df['Cabin'] = df['Cabin'].map({'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0,
#                                          'F': 0, 'G': 0, 'T': 0, 'U': 1}).astype(int)
df['Cabin'].value_counts(normalize=True)

U    0.771044
C    0.066218
B    0.052750
D    0.037037
E    0.035915
A    0.016835
F    0.014590
G    0.004489
T    0.001122
Name: Cabin, dtype: float64

In [7]:
# obtain Title from name (Mr, Mrs, Miss etc)
df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'].value_counts(normalize=True)

Mr          0.580247
Miss        0.204265
Mrs         0.140292
Master      0.044893
Dr          0.007856
Rev         0.006734
Major       0.002245
Mlle        0.002245
Col         0.002245
Capt        0.001122
Ms          0.001122
Lady        0.001122
Don         0.001122
Sir         0.001122
Mme         0.001122
Countess    0.001122
Jonkheer    0.001122
Name: Title, dtype: float64

In [8]:
# concolidate titles
df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Dona'],'Royalty')
df['Title'] = df['Title'].replace(['Mme'], 'Mrs')
df['Title'] = df['Title'].replace(['Mlle','Ms'], 'Miss')
df['Title'] = df['Title'].replace(['Capt', 'Col', 'Major','Rev'], 'Officer')
df['Title'] = df['Title'].replace(['Jonkheer', 'Don','Sir'], 'Royalty')
# dataset.loc[(dataset.Sex == 'male')   & (dataset.Title == 'Dr'),'Title'] = 'Mr'
# dataset.loc[(dataset.Sex == 'female') & (dataset.Title == 'Dr'),'Title'] = 'Mrs'
df['Title'].value_counts(normalize=True)

Mr         0.580247
Miss       0.207632
Mrs        0.141414
Master     0.044893
Officer    0.012346
Dr         0.007856
Royalty    0.005612
Name: Title, dtype: float64

In [9]:
missing = df.copy().isna().sum()
(missing.reset_index(name='cnt')
 .sort_values(by='cnt', ascending=True))

,index,cnt
0,PassengerId,0
1,Survived,0
2,Pclass,0
3,Name,0
4,Sex,0
6,SibSp,0
7,Parch,0
8,Ticket,0
9,Fare,0
10,Cabin,0


In [10]:
df['Age'] = df['Age'].fillna(0)
df.Age = df.Age.astype(float)
df['Age'].value_counts(normalize=True).head()

0.0     0.198653
24.0    0.033670
22.0    0.030303
18.0    0.029181
28.0    0.028058
Name: Age, dtype: float64

In [11]:
df[df.Age<0]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title


In [12]:
gpd = df[df.Age>0]
gpd = (gpd.groupby(['Title', 'Sex'])['Age']
 .mean()
 .to_frame()
 .reset_index()
#  .rename(columns={0:'cnt'})
 .sort_values(by=['Title', 'Sex'], ascending=False)
).round(0).set_index(['Title', 'Sex'])
gpd

Age
Title   Sex         
Royalty male    42.0
        female  40.0
Officer male    49.0
Mrs     female  36.0
Mr      male    32.0
Miss    female  22.0
Master  male     5.0
Dr      male    41.0
        female  49.0

In [13]:
d = dict(zip(gpd.index, gpd.Age))
d

{('Royalty', 'male'): 42.0,
 ('Royalty', 'female'): 40.0,
 ('Officer', 'male'): 49.0,
 ('Mrs', 'female'): 36.0,
 ('Mr', 'male'): 32.0,
 ('Miss', 'female'): 22.0,
 ('Master', 'male'): 5.0,
 ('Dr', 'male'): 41.0,
 ('Dr', 'female'): 49.0}

In [14]:
conditions  = conditions2 = [(df.Title == k[0]) & (df.Sex == k[1]) & (df.Age == 0) for k in d.keys()]
choices     = [v for v in d.values()]

df['Age2'] = np.select(conditions, choices, default = df['Age']).round(0)
df.sample(10)
# df[df.Title=='Master']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Age2
687,688,0,3,"Dakic, Mr. Branko",male,19.0,0,0,349228,10.1708,U,S,Mr,19.0
167,168,0,3,"Skoog, Mrs. William (Anna Bernhardina Karlsson)",female,45.0,1,4,347088,27.9000,U,S,Mrs,45.0
180,181,0,3,"Sage, Miss. Constance Gladys",female,0.0,8,2,CA. 2343,69.5500,U,S,Miss,22.0
472,473,1,2,"West, Mrs. Edwy Arthur (Ada Mary Worth)",female,33.0,1,2,C.A. 34651,27.7500,U,S,Mrs,33.0
614,615,0,3,"Brocklebank, Mr. William Alfred",male,35.0,0,0,364512,8.0500,U,S,Mr,35.0
599,600,1,1,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",male,49.0,1,0,PC 17485,56.9292,A,C,Royalty,49.0
485,486,0,3,"Lefebre, Miss. Jeannie",female,0.0,3,1,4133,25.4667,U,S,Miss,22.0
847,848,0,3,"Markoff, Mr. Marin",male,35.0,0,0,349213,7.8958,U,C,Mr,35.0
419,420,0,3,"Van Impe, Miss. Catharina",female,10.0,0,2,345773,24.1500,U,S,Miss,10.0
270,271,0,1,"Cairns, Mr. Alexander",male,0.0,0,0,113798,31.0000,U,S,Mr,32.0


In [15]:
# Create family size from 'sibsq + parch + 1'
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# (df[['FamilySize', 'Survived']]
# .groupby(['FamilySize'], as_index=False)
#  .mean()
#  .sort_values(by='Survived', ascending=False))

pd.crosstab(df.FamilySize, # ROWS
              df.Survived, # COLS
              values=df.PassengerId, 
              aggfunc=pd.Series.nunique,
              normalize='index',
              dropna=False).round(2)

Survived,0,1
FamilySize,,
1,0.70,0.30
2,0.45,0.55
3,0.42,0.58
4,0.28,0.72
5,0.80,0.20
6,0.86,0.14
7,0.67,0.33
8,1.00,0.00
11,1.00,0.00


In [24]:
# fill the missing values of Embarked feature with the most common occurance

# get most frequent port
most_freq_port = df.Embarked.dropna().mode()[0]

# df['Embarked'] = df['Embarked'].fillna(freq_port)

# inspect
pd.crosstab(df.Embarked, # ROWS
              df.Survived, # COLS
              values=df.PassengerId, 
              aggfunc=pd.Series.nunique,
              normalize='index',
              dropna=False).round(2)


Survived,0,1
Embarked,,
C,0.45,0.55
Q,0.61,0.39
S,0.66,0.34


In [19]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'Age2',
       'FamilySize'],
      dtype='object')

In [20]:
df.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Age2,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,U,S,Mr,22.0,2


In [21]:
missing = df.copy().isna().sum()
(missing.reset_index(name='cnt')
 .sort_values(by='cnt', ascending=True))

,index,cnt
0,PassengerId,0
1,Survived,0
2,Pclass,0
3,Name,0
4,Sex,0
5,Age,0
6,SibSp,0
7,Parch,0
8,Ticket,0
9,Fare,0


In [27]:
cols = ['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age2', 'FamilySize', 'Cabin', 'Embarked']
tmp = df.loc[:, cols].set_index('PassengerId')
tmp.head(1)

,Survived,Pclass,Sex,Age2,FamilySize,Cabin,Embarked
PassengerId,,,,,,,
1,0,3,male,22.0,2,U,S


In [28]:
tmp.to_csv("../local-data/output/titanic_train_clean.csv", index=True)